In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.layers import (Input, BatchNormalization, Cropping3D,
                                     Concatenate, Conv3D, MaxPooling3D,
                                     UpSampling3D, Reshape)
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [8]:
tf.__version__
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
def Unet3dFunctional(img_shape = (320,320,16,2),filters = 64, depth = 3, n_class = 3):
    inputs = Input(shape=img_shape)
    inp = inputs
    residual_connections = []
    for i in range(depth):
        conv = Conv3D(filters=filters,  kernel_size= 3,
                      activation='relu', padding='same')(inp)
        conv = Conv3D(filters=filters,  kernel_size= 3,
                      activation='relu', padding='same')(conv)
        bn = BatchNormalization()(conv)
        inp = MaxPooling3D(pool_size=(2,2,2))(bn)
        filters *=2
        residual_connections.append(bn)

    conv = Conv3D(filters=filters,  kernel_size= 3,
                  activation='relu', padding='same')(inp)
    conv = Conv3D(filters=filters,  kernel_size= 3,
                  activation='relu', padding='same')(conv)
    bn = BatchNormalization()(conv)
    for i in range(depth):
        bn = UpSampling3D(size = (2,2,2))(bn)
        merge = Concatenate(axis=-1)([bn, residual_connections[len(residual_connections)-i-1]])
        filters /=2

        conv = Conv3D(filters=filters,  kernel_size= 3,
                      activation='relu', padding='same')(merge)

        conv = Conv3D(filters=filters,  kernel_size= 3,
                      activation='relu', padding='same')(conv)

        bn = BatchNormalization()(conv)


    out = Conv3D(n_class, 1, activation= 'softmax')(bn)

    model = Model(inputs, out)
    return model

In [10]:
model = Unet3dFunctional(img_shape = (128,128,16,2),filters = 16, depth = 3, n_class = 2)
model.compile(optimizer='sgd',
                       loss = 'categorical_crossentropy',
                       metrics=['accuracy'])

In [11]:
last = model.layers[-1]

In [12]:
last.output

<tf.Tensor 'conv3d_29/Identity:0' shape=(None, 128, 128, 16, 2) dtype=float32>

In [13]:
!pwd

/home/px


In [14]:
%cd MultiPlanarUNet

/home/px/MultiPlanarUNet


In [15]:
seed = 909 # (IMPORTANT) to transform image and corresponding mask with same augmentation parameter.
image_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 #preprocessing_function = image_preprocessing
                                   ) # custom fuction for each image you can use resnet one too.
mask_datagen = ImageDataGenerator(width_shift_range=0.1,
                 height_shift_range=0.1,
                 #preprocessing_function = mask_preprocessing
                                  )  # to make mask as feedable formate (256,256,1)

image_generator =image_datagen.flow_from_directory("data_folder/train/images/",
                                                    class_mode=None, seed=seed)

mask_generator = mask_datagen.flow_from_directory("data_folder/train/labels/",
                                                   class_mode=None, seed=seed)

train_generator = zip(image_generator, mask_generator)



Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [16]:
from mpunet.image.image_pair_loader import ImagePairLoader
imgloader = ImagePairLoader(base_dir="data_folder/train")
imgloader.next()

ImagePairLoader(id=train, images=17, data_dir=/home/px/MultiPlanarUNet/data_folder/train)
--- Image subdir: /home/px/MultiPlanarUNet/data_folder/train/images
--- Label subdir: /home/px/MultiPlanarUNet/data_folder/train/labels


AttributeError: 'ImagePairLoader' object has no attribute 'next'

In [ ]:
import nibabel as nib
import glob
import skimage.transform as skTrans

def generate_3dimage(base_dir="data_folder/train", dim = (320,320,16),
                     batch_size = 2):
    def loadNii(filename):
        img = nib.load(filename)
        data = img.get_fdata()
        return data

    image_base = os.path.join(base_dir, 'images')
    mask_base = os.path.join(base_dir, 'labels')
    files_names = os.listdir(image_base)
    files_names = [i for i in files_names if not i.startswith('.')]
    while True:
        # Select files (paths/indices) for the batch
        batch_names  = np.random.choice(a = files_names, size = batch_size)
        n_channels = 2
        batch_x = np.empty((batch_size, *dim, n_channels))
        batch_y = np.empty((batch_size, *dim))
        # Read in each input, perform preprocessing and get labels
        for i, input_name in enumerate(batch_names):
            image_path = os.path.join(image_base,input_name)
            mask_path = os.path.join(mask_base,input_name)

            input = loadNii(image_path)
            output = loadNii(mask_path)
            input = skTrans.resize(input, dim,
                                   order=1, preserve_range=True)
            output = skTrans.resize(output, dim,
                                   order=1, preserve_range=True)


            batch_x[i] = input
            batch_y[i] = output
        batch_y = np.expand_dims(batch_y, axis=-1)
        yield (batch_x, batch_y)

sb = generate_3dimage(base_dir="data_folder/train", batch_size = 1)

In [ ]:
x, y = next(sb)
y[0].shape

In [ ]:
model = Unet3dFunctional(img_shape = (128,128,16,2),filters = 16, depth = 3, n_class = 2)
model.compile(optimizer='sgd',
                       loss = 'categorical_crossentropy',
                       metrics=['accuracy'])

model.fit_generator(sb, epochs=5)

In [ ]:
import numpy as np
import tensorflow.keras
import nibabel as nib
import glob
import skimage.transform as skTrans
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, base_dir="data_folder/train", labels=1, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        #self.labels = labels
        self.image_base = os.path.join(base_dir, 'images')
        self.mask_base = os.path.join(base_dir, 'labels')
        files_names = os.listdir(self.image_base)
        files_names = [i for i in files_names if not i.startswith('.')]
        self.files_names = files_names
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.files_names) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        cur_files_names = self.files_names[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(cur_files_names)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        #self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.files_names)

    def __data_generation(self, cur_files_names):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty(self.batch_size, *self.dim)

        for i, input_name in enumerate(cur_files_names):
            image_path = os.path.join(self.image_base,cur_files_names)
            mask_path = os.path.join(self.mask_base,cur_files_names)

            input = nib.load(image_path).get_fdata()
            output = nib.load(mask_path).get_fdata()
            input = skTrans.resize(input, *self.dim,
                                   order=1, preserve_range=True)
            output = skTrans.resize(output, *self.dim,
                                   order=1, preserve_range=True)


            X[i] = input
            y[i] = output


        return X, y

In [ ]:
dataset = tf.data.Dataset.from_generator(gen, (tf.dtypes.float32, tf.dtypes.int32))

In [ ]:
dataset = tf.data.Dataset.from_generator(
    generator=sb,
    output_types=(np.float32, np.int32),
)

In [ ]:
sb = DataGenerator(base_dir="data_folder/train", batch_size=6, dim=(320,320,20))
sb.next()

In [ ]:
class Unet3D(Model):
    def __init__(self,original_dim,filters=64, depth=4, n_class=3,
                 padding='same', name='auto_encoder',**kwargs):
        super().__init__(name = name,**kwargs)
        self.filters = filters
        self.original_dim = original_dim
        self.conv = Conv3D(filters=filters,  kernel_size= 3,
                  activation='relu', padding='same')
        self.n_class = n_class
        self.depth = depth
    # def build(self,input_shape):
    #     super().build(input_shape = input_shape)

    def call(self, inputs):
        inp = inputs
        filters = self.filters
        residual_connections = []
        n_class = self.n_class
        depth = self.depth
        for i in range(4):
            conv = Conv3D(filters=filters,  kernel_size= 3,
                          activation='relu', padding='same')(inp)
            conv = Conv3D(filters=filters,  kernel_size= 3,
                          activation='relu', padding='same')(conv)
            bn = BatchNormalization()(conv)
            inp = MaxPooling3D(pool_size=(2,2,2))(bn)
            filters *=2
            residual_connections.append(bn)
        conv = Conv3D(filters=filters,  kernel_size= 3,
              activation='relu', padding='same')(inp)
        conv = Conv3D(filters=filters,  kernel_size= 3,
                      activation='relu', padding='same')(conv)
        bn = BatchNormalization()(conv)

        for i in range(depth):
            bn = UpSampling3D(size = (2,2,2))(bn)
            merge = Concatenate(axis=-1)([bn, residual_connections[len(residual_connections)-i-1]])
            filters /=2

            conv = Conv3D(filters=filters,  kernel_size= 3,
                          activation='relu', padding='same')(merge)

            conv = Conv3D(filters=filters,  kernel_size= 3,
                          activation='relu', padding='same')(conv)

            bn = BatchNormalization()(conv)


        out = Conv3D(n_class, 1, activation= 'softmax')(bn)
        # ce_loss = tf.keras.losses.CategoricalCrossentropy()
        # self.add_loss(ce_loss)
        # self.add_metric(ce_loss, name="ce_loss")

        return out
unet = Unet3D(original_dim=(224,224,224,3),filters=64)

In [ ]:
unet.build((None, 224,224,224,3))

In [ ]:
unet.summary()

In [ ]:
model_ = Unet3D(original_dim=(224,224,224,3))
inputs = tf.keras.Input(shape=(224,224,224,3,))
outputs = model_(inputs)
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.summary()